In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline

In [2]:
import os.path
pbf_file_path = '../input_static/germany-latest.osm.pbf'
use_queries = not os.path.exists(pbf_file_path)

In [3]:
import pandas as pd
import numpy as np
from quetzal.model import stepmodel
from tqdm import tqdm

# If you go the pbf-way, you need "pyrosm"
if use_queries:
    from osmnx.features import features_from_polygon as get_pois
    #import requests
    #import json
else:
    from pyrosm import OSM

In [4]:
model_path = '../model/' + scenario + '/'
input_path = '../input/'

# Number of POIs from OSM data

There are two ways of fetching data from OpenStreetmap:
* Using overpass queries
* Using locally saved osm.pbf file

Overpass queries take a long time, but work without any further packages. The faster version requires downloading a pbf file for the region of Germany. This is available here (should be saved under `input_static`: https://download.geofabrik.de/europe.html

The extraction from OpenStreetmap makes use of Overpass querys (for documentation see http://osmlab.github.io/learnoverpass/en/docs/). <br>
For trying out querys and their results, use https://overpass-turbo.eu/ (although a success in overpass turbo does not always imply a success in my request). 

For documentation of OpenStreetMap data, see OSM Wiki: https://wiki.openstreetmap.org/.

In [5]:
sm = stepmodel.read_json(model_path + 'de_zones')

In [6]:
sm.zones.sample()

,ARS,NUTS_ID,name,area,population,lat,lon,urbanisation,pop_share,FID,car_avail_hh,employed,employment,geometry,zip_code
index,,,,,,,,,,,,,,,
55700016,55700016,DEA38,Drensteinfurt,106.6,15540,7.741663,51.797048,3,0.056017,55700016,0.948745,7409.896293,76.409896,"POLYGON ((7.67041 51.87676, 7.67613 51.87657, ...",[48317]


In [7]:
# Load POI code Excel
poi_codes = pd.read_excel(input_path + 'spatial_OSM_POI_list.xlsx')

In [10]:
# The OSMNx way
if use_queries:
    missing_values = []
    # define POI dicts
    poi_values = poi_codes.loc[poi_codes['key'].notna()
                              ].drop_duplicates(subset=['key', 'value'], keep='first'
                                               ).set_index('key')['value'].fillna(True)
    poi_keys = [k+' '+v if isinstance(v, str) else k for k,v in poi_values.items()]
    poi_dicts = dict(zip(poi_keys, [{k:v} for k,v in poi_values.items()]))
    
    # Create POI DataFrame
    num_pois = pd.DataFrame(columns=poi_dicts.keys(), index=sm.zones.index)
    name_pois = pd.DataFrame(columns=poi_dicts.keys(), index=sm.zones.index)
    
    # Fill DataFrame with POIs
    for zone, geo in tqdm(sm.zones['geometry'].items(), total=len(sm.zones)):
        for poi, custom_filter in poi_dicts.items():
            try:
                pois = get_pois(geo, custom_filter)
                num_pois.at[zone, poi] = len(pois)
            except:
                missing_values.append([zone, poi])
            try:
                name_pois.at[zone, poi] = list(pois['name'])
            except KeyError:
                name_pois.at[zone, poi] = []

  0%|▎                                                                            | 18/4602 [23:16<96:47:44, 76.02s/it]C:\Users\m.arnz\AppData\Local\miniforge-pypy3\envs\quetzal\lib\site-packages\osmnx\features.py:1047: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  gdf = gdf.dropna(axis="columns", how="all")
  0%|▎                                                                            | 21/4602 [27:10<99:21:52, 78.09s/it]C:\Users\m.arnz\AppData\Local\miniforge-pypy3\envs\quetzal\lib\site-packages\osmnx\features.py:1047: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  gdf = gdf.dropna(axis="columns", how="all")
C:\Users\m.arnz\AppData\Local\miniforge-pypy3\envs\quetzal\lib\site-packages\osmnx\features.py:1047: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  gdf = gdf.dropna(axis="col

  2%|█▏                                                                       | 36/2225 [5:44:58<100:42:51, 165.63s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
  2%|█▏                                                                        | 37/2225 [5:47:19<96:06:34, 158.13s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

  2%|█▊                                                                         | 53/2225 [6:08:32<46:21:06, 76.83s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be r

  3%|██▎                                                                        | 70/2225 [6:29:53<42:56:48, 71.74s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
  3%|██▍                                                                        | 71/2225 [6:30:46<39:30:34, 66.03s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

  4%|██▋                                                                       | 80/2225 [6:47:31<66:30:42, 111.63s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
  4%|██▋                                                                       | 81/2225 [6:50:16<75:56:28, 127.51s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

...

 95%|████████████████████████████████████████████████████████████████████▉    | 2103/2225 [66:11:21<1:49:32, 53.87s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
 95%|█████████████████████████████████████████████████████████████████████    | 2104/2225 [66:12:28<1:56:06, 57.57s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

 95%|█████████████████████████████████████████████████████████████████████▎   | 2114/2225 [66:26:06<2:36:01, 84.34s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
 95%|█████████████████████████████████████████████████████████████████████▍   | 2117/2225 [66:29:23<2:08:32, 71.41s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

 96%|█████████████████████████████████████████████████████████████████████▉   | 2131/2225 [66:44:20<1:43:57, 66.36s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
 96%|█████████████████████████████████████████████████████████████████████▉   | 2133/2225 [66:47:10<1:58:13, 77.11s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

 97%|██████████████████████████████████████████████████████████████████████▏ | 2169/2225 [67:34:00<1:35:58, 102.84s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
 98%|███████████████████████████████████████████████████████████████████████▏ | 2171/2225 [67:36:03<1:12:43, 80.81s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

100%|██████████████████████████████████████████████████████████████████████████▋| 2214/2225 [68:31:31<10:09, 55.43s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
100%|██████████████████████████████████████████████████████████████████████████▋| 2216/2225 [68:33:38<09:00, 60.06s/it]C:\Users\marlin.arnz\.conda\envs\quetzal\lib\site-pac

In [23]:
# Add missing values
if use_queries:
    for zone, poi in tqdm(missing_values):
        try:
            pois = get_pois(sm.zones.loc[zone, 'geometry'], poi_dicts[poi])
            num_pois.at[zone, poi] = len(pois)
            name_pois.at[zone, poi] = list(pois['name'])
        except:
            num_pois.at[zone, poi] = 0
            name_pois.at[zone, poi] = []

  1%|▊                                                                         | 1426/133456 [02:02<3:00:18, 12.20it/s]C:\Users\m.arnz\AppData\Local\miniforge-pypy3\envs\quetzal\lib\site-packages\osmnx\features.py:1047: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  gdf = gdf.dropna(axis="columns", how="all")
  1%|▉                                                                         | 1763/133456 [02:40<3:27:36, 10.57it/s]C:\Users\m.arnz\AppData\Local\miniforge-pypy3\envs\quetzal\lib\site-packages\osmnx\features.py:1047: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  gdf = gdf.dropna(axis="columns", how="all")
100%|████████████████████████████████████████████████████████████████████████| 133456/133456 [3:05:02<00:00, 12.02it/s]


In [ ]:
# The Pyrosm way using local pbf file of Germany
if not use_queries:
    # define POI dicts
    poi_values = poi_codes.loc[poi_codes['key'].notna()
                              ].drop_duplicates(subset=['key', 'value'], keep='first'
                                               ).set_index('key')['value'].fillna(True)
    poi_keys = [k+' '+v if isinstance(v, str) else k for k,v in poi_values.items()]
    poi_dicts = dict(zip(poi_keys, [{k:v} for k,v in poi_values.items()]))
    
    # Create POI DataFrame
    num_pois = pd.DataFrame(columns=poi_dicts.keys(), index=sm.zones.index)
    name_pois = pd.DataFrame(columns=poi_dicts.keys(), index=sm.zones.index)
    
    # Fill DataFrame with POIs
    for zone, geo in tqdm(sm.zones['geometry'].items()):
        osm = OSM(pbf_file_path, bounding_box=geo)
        for poi, custom_filter in poi_dicts.items():
            pois = osm.get_pois(custom_filter=custom_filter)
            num_pois.at[zone, poi] = len(pois)
            name_pois.at[zone, poi] = list(pois['name'])

In [24]:
num_pois.sample(3)

,amenity college,amenity school,amenity university,building school,building university,craft,amenity kindergarten,building retail,amenity marketplace,shop bakery,...,leisure water_park,tourism aquarium,tourism attraction,tourism camp_site,tourism caravan_site,tourism hostel,tourism hotel,tourism theme_park,tourism zoo,water lake
index,,,,,,,,,,,,,,,,,,,,,
91815138,0,4,0,1,0,0,0,2,0,4,...,0,0,0,0,0,1,2,0,0,0
81350020,1,7,0,6,0,12,10,15,1,9,...,1,0,2,0,2,0,5,0,0,2
94720150,0,1,0,2,0,2,2,9,0,2,...,0,0,0,1,0,0,1,0,0,0


In [25]:
assert len(num_pois.loc[num_pois.isna().any(axis=1)]) == 0

In [26]:
# Saving table
num_pois.to_csv(input_path + 'spatial_num_pois_raw.csv')

In [13]:
# Saving table
#name_pois.to_csv(input_path + 'spatial_name_pois_raw.csv')